In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
os.chdir("../")
import random
import xgboost
import time
import numpy as np
from utils.metrics import PEHE, ATE

# Random generators initialization
seed=42
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)

In [2]:
# Start timer
start = time.time()

# Load train data
data = np.load('Data/train.npz')
trainX, trainT, trainY, train_potential_Y = data['X'], data['T'], data['Y'], data['potential_Y']
# Load valid data
data = np.load('Data/test.npz')
testX, testT, testY, test_potential_Y = data['X'], data['T'], data['Y'], data['potential_Y']
print("[INFO] Dataset imported")
print(f"[INFO] Number of training instances: {trainX.shape[0]}")
print(f"[INFO] Number of testing instances: {testX.shape[0]}")

    

# Loss function
# -----------------------------------------------------------------------------------------------------
treatment = np.array([[1,0] if x == 0 else [0,1] for x in trainT]).flatten()

def custom_loss(y_true:np.ndarray=None, y_pred:np.ndarray=None)->(np.ndarray, np.ndarray):
    grad = 2*(y_pred.flatten() - y_true.flatten()) * treatment
    hess = (0*y_pred.flatten() + 2)  * treatment

    return grad, hess
    


# Setup XGBoost
# -----------------------------------------------------------------------------------------------------
model = xgboost.XGBRegressor(n_estimators=500, 
                                max_depth=4, 
                                objective=custom_loss, 
                                learning_rate=1e-2, 
                            #  booster="gblinear",
                                n_jobs=-1,
                                tree_method="hist", 
                                multi_strategy="multi_output_tree")
    
# Create outputs for DragonNet (concatenate Y & T)
yt_train = np.concatenate([trainY.reshape(-1,1), trainT.reshape(-1,1)], axis = 1)


# Train model
model.fit(trainX, yt_train, eval_set=[(trainX, train_potential_Y), (testX, test_potential_Y)], verbose=25);
print('[INFO] Model trained')    
print('[INFO] Time %.2f seconds' % (time.time() - start))

[INFO] Dataset imported
[INFO] Number of training instances: 5000
[INFO] Number of testing instances: 1000
[0]	validation_0-rmse:0.49688	validation_1-rmse:0.49679
[25]	validation_0-rmse:0.43347	validation_1-rmse:0.43116
[50]	validation_0-rmse:0.39287	validation_1-rmse:0.38861
[75]	validation_0-rmse:0.36818	validation_1-rmse:0.36219
[100]	validation_0-rmse:0.35408	validation_1-rmse:0.34675
[125]	validation_0-rmse:0.34655	validation_1-rmse:0.33828
[150]	validation_0-rmse:0.34292	validation_1-rmse:0.33400
[175]	validation_0-rmse:0.34150	validation_1-rmse:0.33218
[200]	validation_0-rmse:0.34122	validation_1-rmse:0.33175
[225]	validation_0-rmse:0.34154	validation_1-rmse:0.33203
[250]	validation_0-rmse:0.34203	validation_1-rmse:0.33264
[275]	validation_0-rmse:0.34249	validation_1-rmse:0.33332
[300]	validation_0-rmse:0.34295	validation_1-rmse:0.33403
[325]	validation_0-rmse:0.34335	validation_1-rmse:0.33469
[350]	validation_0-rmse:0.34359	validation_1-rmse:0.33528
[375]	validation_0-rmse:0.34

In [3]:
# Get predictions
# -----------------------------------------------------------------------------------------------------
test_y_hat = model.predict(testX)

# Calculate performance metrics
# -----------------------------------------------------------------------------------------------------
# ATE
real_ATE = ( test_potential_Y[:,1] - test_potential_Y[:,0] ).mean()      
# Error ATE
Error_ATE = ATE(test_potential_Y, test_y_hat)  
# PEHE
PEHE_score = PEHE(test_potential_Y, test_y_hat)

print(f"Error_ATE: {Error_ATE:.3f}")
print(f"PEHE: {PEHE_score:.3f}")

Error_ATE: 0.095
PEHE: 0.230
